In [64]:
import os
import json
from datetime import datetime
from typing import Dict, List, Optional
import requests

In [65]:
os.environ['HF_TOKEN']=''

In [66]:
HF_API_URL = "https://router.huggingface.co/v1"

In [67]:
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
GENERATION_MODEL = "mistralai/Mistral-7B-Instruct-v0.2:featherless-ai"

In [68]:
DISEASE_KNOWLEDGE_BASE = {
    "tomato_early_blight": {
        "cause": "nitrogen deficiency and fungal infection (Alternaria solani)",
        "soil_relation": "depleted nitrogen levels in soil",
        "symptoms": "dark spots on leaves, yellowing"
    },
    "tomato_late_blight": {
        "cause": "fungal infection (Phytophthora infestans) and high moisture",
        "soil_relation": "poor drainage and excess moisture retention",
        "symptoms": "water-soaked lesions, white mold"
    },
    "tomato_leaf_mold": {
        "cause": "fungal infection due to high humidity",
        "soil_relation": "poor air circulation and waterlogged soil",
        "symptoms": "yellow spots on upper leaf surface"
    },
    "tomato_septoria_leaf_spot": {
        "cause": "fungal infection (Septoria lycopersici)",
        "soil_relation": "nitrogen imbalance and wet conditions",
        "symptoms": "small circular spots with dark borders"
    },
    "tomato_bacterial_spot": {
        "cause": "bacterial infection (Xanthomonas spp.)",
        "soil_relation": "poor soil pH and nutrient imbalance",
        "symptoms": "dark greasy spots on leaves and fruits"
    },
    "rice_blast": {
        "cause": "fungal infection (Magnaporthe oryzae)",
        "soil_relation": "excessive nitrogen and poor silicon levels",
        "symptoms": "diamond-shaped lesions"
    },
    "rice_brown_spot": {
        "cause": "fungal infection and nutrient deficiency",
        "soil_relation": "potassium and silicon deficiency",
        "symptoms": "brown spots on leaves"
    },
    "wheat_rust": {
        "cause": "fungal infection (Puccinia spp.)",
        "soil_relation": "nitrogen excess and phosphorus deficiency",
        "symptoms": "orange-red pustules on leaves"
    },
    "potato_early_blight": {
        "cause": "fungal infection (Alternaria solani)",
        "soil_relation": "nitrogen and calcium deficiency",
        "symptoms": "concentric rings on leaves"
    },
    "corn_common_rust": {
        "cause": "fungal infection (Puccinia sorghi)",
        "soil_relation": "zinc and manganese deficiency",
        "symptoms": "reddish-brown pustules"
    }
}

In [ ]:
EXTENDED_DISEASE_KB = {
    "potato_late_blight": {
        "cause": "Phytophthora infestans, favored by cool moist conditions",
        "soil_relation": "poor drainage and excess nitrogen",
        "symptoms": "water-soaked lesions, white fungal growth"
    },
    "grape_black_rot": {
        "cause": "Guignardia bidwellii fungal infection",
        "soil_relation": "calcium and potassium deficiency",
        "symptoms": "brown lesions with black dots"
    },
    "cotton_bacterial_blight": {
        "cause": "Xanthomonas bacteria",
        "soil_relation": "nitrogen excess and poor pH",
        "symptoms": "angular leaf spots, boll rot"
    },
    "wheat_leaf_rust": {
        "cause": "Puccinia triticina fungus",
        "soil_relation": "phosphorus deficiency",
        "symptoms": "orange pustules on leaves"
    },
    "maize_leaf_blight": {
        "cause": "Exserohilum turcicum fungus",
        "soil_relation": "nitrogen imbalance",
        "symptoms": "long tan lesions with dark borders"
    },
    "soybean_rust": {
        "cause": "Phakopsora pachyrhizi fungus",
        "soil_relation": "potassium deficiency",
        "symptoms": "small tan to dark brown lesions"
    },
    "apple_scab": {
        "cause": "Venturia inaequalis fungus",
        "soil_relation": "poor soil aeration",
        "symptoms": "olive-green spots on leaves and fruit"
    },
    "citrus_canker": {
        "cause": "Xanthomonas citri bacteria",
        "soil_relation": "copper and zinc deficiency",
        "symptoms": "raised brown lesions with yellow halos"
    }
}
DISEASE_KNOWLEDGE_BASE.update(EXTENDED_DISEASE_KB)

In [69]:
SOIL_RECOMMENDATIONS = {
    "red_soil": {
        "characteristics": "iron-rich, acidic, low nitrogen and phosphorus",
        "amendments": "add organic matter, lime for pH, NPK fertilizers"
    },
    "black_soil": {
        "characteristics": "high clay content, moisture-retentive, rich in calcium",
        "amendments": "gypsum for structure, drainage improvement"
    },
    "alluvial_soil": {
        "characteristics": "fertile, balanced nutrients, good drainage",
        "amendments": "maintain organic matter, balanced NPK"
    },
    "laterite_soil": {
        "characteristics": "iron and aluminum-rich, acidic, low fertility",
        "amendments": "heavy organic matter, lime, phosphate fertilizers"
    },
    "desert_soil": {
        "characteristics": "sandy, low organic matter, poor water retention",
        "amendments": "compost, mulching, drip irrigation"
    },
    "mountain_soil": {
        "characteristics": "acidic, high organic matter, prone to erosion",
        "amendments": "erosion control, pH adjustment, balanced nutrients"
    }
}


In [70]:
GOVERNMENT_SCHEMES_BY_STATE = {
    "andhra_pradesh": [
        {"name": "YSR Rythu Bharosa", "subsidy": "up to 13,500 per year", "type": "financial_support"},
        {"name": "PM-KISAN", "subsidy": "6,000 per year", "type": "direct_benefit"},
        {"name": "Soil Health Card Scheme", "subsidy": "free soil testing", "type": "soil_management"}
    ],
    "telangana": [
        {"name": "Rythu Bandhu", "subsidy": "10,000 per acre per season", "type": "financial_support"},
        {"name": "PM-KISAN", "subsidy": "6,000 per year", "type": "direct_benefit"},
        {"name": "Rythu Bima", "subsidy": "life insurance for farmers", "type": "insurance"}
    ],
    "karnataka": [
        {"name": "Krishi Bhagya", "subsidy": "up to 10,000 per hectare", "type": "financial_support"},
        {"name": "PM-KISAN", "subsidy": "6,000 per year", "type": "direct_benefit"},
        {"name": "Subsidized Fertilizer Scheme", "subsidy": "50% subsidy on fertilizers", "type": "input_subsidy"}
    ],
    "tamil_nadu": [
        {"name": "Uzhavar Sandhai", "subsidy": "direct market access", "type": "market_support"},
        {"name": "PM-KISAN", "subsidy": "6,000 per year", "type": "direct_benefit"},
        {"name": "Free Electricity for Agriculture", "subsidy": "free power supply", "type": "utility_support"}
    ],
    "maharashtra": [
        {"name": "Mahatma Jyotiba Phule Shetkari Karjmukti Yojana", "subsidy": "loan waiver", "type": "debt_relief"},
        {"name": "PM-KISAN", "subsidy": "6,000 per year", "type": "direct_benefit"},
        {"name": "Soil Health Management", "subsidy": "75% subsidy on soil amendments", "type": "input_subsidy"}
    ],
    "punjab": [
        {"name": "PM-KISAN", "subsidy": "6,000 per year", "type": "direct_benefit"},
        {"name": "Crop Residue Management", "subsidy": "up to 50% on machinery", "type": "equipment_subsidy"},
        {"name": "Water Conservation Scheme", "subsidy": "subsidy on drip irrigation", "type": "irrigation_support"}
    ],
    "haryana": [
        {"name": "Meri Fasal Mera Byora", "subsidy": "crop insurance and MSP", "type": "insurance"},
        {"name": "PM-KISAN", "subsidy": "6,000 per year", "type": "direct_benefit"},
        {"name": "Bhavantar Bharpayee Yojana", "subsidy": "price deficiency payment", "type": "price_support"}
    ],
    "uttar_pradesh": [
        {"name": "Kisan Samman Nidhi", "subsidy": "6,000 per year", "type": "direct_benefit"},
        {"name": "Mukhyamantri Kisan Evam Sarvhit Bima Yojana", "subsidy": "insurance coverage", "type": "insurance"},
        {"name": "Organic Farming Scheme", "subsidy": "50% subsidy on organic inputs", "type": "input_subsidy"}
    ],
    "west_bengal": [
        {"name": "Krishak Bandhu", "subsidy": "5,000 per acre per year", "type": "financial_support"},
        {"name": "PM-KISAN", "subsidy": "6,000 per year", "type": "direct_benefit"},
        {"name": "Crop Insurance Scheme", "subsidy": "premium subsidy", "type": "insurance"}
    ],
    "bihar": [
        {"name": "PM-KISAN", "subsidy": "6,000 per year", "type": "direct_benefit"},
        {"name": "Bihar Rajya Fasal Sahayata Yojana", "subsidy": "crop loss compensation", "type": "insurance"},
        {"name": "Diesel Anudan Yojana", "subsidy": "subsidy on diesel for farming", "type": "fuel_subsidy"}
    ],
    "madhya_pradesh": [
        {"name": "PM-KISAN", "subsidy": "6,000 per year", "type": "direct_benefit"},
        {"name": "Mukhyamantri Kisan Kalyan Yojana", "subsidy": "4,000 per year additional", "type": "financial_support"},
        {"name": "Bhavantar Bhugtan Yojana", "subsidy": "price compensation", "type": "price_support"}
    ],
    "rajasthan": [
        {"name": "PM-KISAN", "subsidy": "6,000 per year", "type": "direct_benefit"},
        {"name": "Rajasthan Crop Insurance Scheme", "subsidy": "premium support", "type": "insurance"},
        {"name": "Soil Testing Subsidy", "subsidy": "free soil testing", "type": "soil_management"}
    ],
    "gujarat": [
        {"name": "PM-KISAN", "subsidy": "6,000 per year", "type": "direct_benefit"},
        {"name": "Mukhyamantri Kisan Sahay Yojana", "subsidy": "crop loss compensation", "type": "insurance"},
        {"name": "Irrigation Subsidy", "subsidy": "50% on drip/sprinkler systems", "type": "irrigation_support"}
    ],
    "odisha": [
        {"name": "KALIA Scheme", "subsidy": "up to 25,000 per family", "type": "financial_support"},
        {"name": "PM-KISAN", "subsidy": "6,000 per year", "type": "direct_benefit"},
        {"name": "Balaram Yojana", "subsidy": "interest-free crop loans", "type": "credit_support"}
    ],
    "assam": [
        {"name": "PM-KISAN", "subsidy": "6,000 per year", "type": "direct_benefit"},
        {"name": "Mukhyamantri Krishi Sa-Sajuli Yojana", "subsidy": "90% subsidy on farm equipment", "type": "equipment_subsidy"},
        {"name": "Organic Farming Mission", "subsidy": "support for organic inputs", "type": "input_subsidy"}
    ],
    "kerala": [
        {"name": "PM-KISAN", "subsidy": "6,000 per year", "type": "direct_benefit"},
        {"name": "Agricultural Workers Welfare Fund", "subsidy": "financial assistance", "type": "welfare"},
        {"name": "Vegetable Development Scheme", "subsidy": "50% subsidy on inputs", "type": "input_subsidy"}
    ],
    "delhi": [
        {"name": "PM-KISAN", "subsidy": "6,000 per year", "type": "direct_benefit"},
        {"name": "Agricultural Machinery Subsidy", "subsidy": "40-50% on equipment", "type": "equipment_subsidy"}
    ]
}

In [71]:
WEED_MANAGEMENT = {
    "low_cost": [
        "Manual weeding using hand tools (khurpi, kassi)",
        "Mulching with crop residue or dried leaves",
        "Inter-cropping to suppress weed growth",
        "Timely hoeing between rows",
        "Use of black plastic mulch in small patches"
    ],
    "high_cost": [
        "Selective pre-emergence herbicides (Pendimethalin)",
        "Post-emergence herbicides (2,4-D, Atrazine)",
        "Mechanical weeders (power weeder, rotary weeder)",
        "Integrated Weed Management (IWM) with bio-herbicides",
        "Weed fabric and automated weeding systems"
    ]
}

In [72]:
def call_huggingface_api(payload) -> Dict:
    headers = {"Authorization": f"Bearer {os.environ['HF_TOKEN']}"}
    url = "https://router.huggingface.co/v1/chat/completions"

    try:
        response = requests.post(url, headers=headers, json=payload, timeout=30)
        return response.json()
    except Exception as e:
        print(f"API Error: {str(e)}")
        return {"error": str(e)}

In [73]:
def query(payload):
  API_URL = "https://router.huggingface.co/hf-inference/models/sentence-transformers/all-MiniLM-L6-v2/pipeline/sentence-similarity"
  headers = {
    "Authorization": f"Bearer {os.environ['HF_TOKEN']}",
  }
  try:
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()
  except Exception as e:
    print(f"API Error: {str(e)}")
    return {"error": str(e)}

In [74]:
def get_embeddings(text: str) -> List[float]:
    payload = {"inputs": text}
    result = query(payload)

    return result[0]

In [75]:
def generate_text_with_llm(prompt: str, max_tokens: int = 500) -> str:
    payload = {
        "messages": [
            {
                "role": "user",
                "content": f"{prompt}"
            }
        ],
        "model": "mistralai/Mistral-7B-Instruct-v0.2:featherless-ai",
        "parameters": {
            "max_new_tokens": max_tokens,
            "temperature": 0.7,
            "top_p": 0.9,
            "do_sample": True
        }
    }

    result = call_huggingface_api(payload)

    if isinstance(result, list) and len(result) > 0:
        return result[0].get("generated_text", "")
    elif isinstance(result,Dict) and len(result) > 0:
        return result["choices"][0]["message"]['content']

    return "Unable to generate response. Please check API configuration."

In [76]:
def get_user_input() -> Dict:
    state = input("\nEnter your state (e.g., punjab, karnataka): ").lower().strip()
    budget = float(input("Enter your budget (in INR): "))

    print("\nIs this input from automatic prediction? (yes/no)")
    auto_input = input().lower().strip() == "yes"

    if auto_input:
        print("\n--- Automatic Input Mode ---")
        weed_detected = input("Weed detected (yes/no): ").lower().strip() == "yes"
        soil_type = input("Soil type: ").lower().strip()
        disease = input("Disease name (or 'none'): ").lower().strip()
    else:
        print("\n--- Manual Input Mode ---")
        weed_detected = input("Is weed present? (yes/no): ").lower().strip() == "yes"
        soil_type = input("Enter soil type (red_soil/black_soil/alluvial_soil/laterite_soil/desert_soil/mountain_soil): ").lower().strip()
        disease = input("Enter disease name (or 'none' if no disease): ").lower().strip()

    if disease == "none" or disease == "":
        disease = None

    return {
        "state": state,
        "budget": budget,
        "weed_detected": weed_detected,
        "soil_type": soil_type,
        "disease": disease,
        "input_mode": "automatic" if auto_input else "manual",
        "timestamp": datetime.now().isoformat()
    }


In [90]:
def retrieve_disease_context(disease: str) -> Dict:
    disease_key = disease.lower().replace(" ", "_")

    if disease_key in DISEASE_KNOWLEDGE_BASE:
        return DISEASE_KNOWLEDGE_BASE[disease_key]
    if disease:
        disease_embedding = get_embeddings(disease)
        best_match = None
        best_score = -1

        for key in DISEASE_KNOWLEDGE_BASE.keys():
            key_embedding = get_embeddings(key)
            if disease_embedding and key_embedding:
                score = sum(a*b for a, b in zip(disease_embedding, key_embedding))
                if score > best_score:
                    best_score = score
                    best_match = key

        if best_match and best_score > 0.5:
            return DISEASE_KNOWLEDGE_BASE[best_match]

    return {
        "cause": "unknown disease",
        "soil_relation": "general nutrient deficiency",
        "symptoms": "varied symptoms"
    }
disease_info=retrieve_disease_context("rice_blast")

In [91]:
def retrieve_soil_context(soil_type: str) -> Dict:
    soil_key = soil_type.lower().replace(" ", "_")

    if soil_key in SOIL_RECOMMENDATIONS:
        return SOIL_RECOMMENDATIONS[soil_key]

    return {
        "characteristics": "general soil properties",
        "amendments": "balanced NPK fertilizer and organic matter"
    }
soil_info=retrieve_soil_context("red_soil")

In [81]:
def get_government_schemes(state: str) -> List[Dict]:
    state_key = state.lower().replace(" ", "_")

    if state_key in GOVERNMENT_SCHEMES_BY_STATE:
        return GOVERNMENT_SCHEMES_BY_STATE[state_key]

    # Return default PM-KISAN if state not found
    return [{"name": "PM-KISAN", "subsidy": "6,000 per year", "type": "direct_benefit"}]
get_government_schemes("karnataka")

[{'name': 'Krishi Bhagya',
  'subsidy': 'up to 10,000 per hectare',
  'type': 'financial_support'},
 {'name': 'PM-KISAN', 'subsidy': '6,000 per year', 'type': 'direct_benefit'},
 {'name': 'Subsidized Fertilizer Scheme',
  'subsidy': '50% subsidy on fertilizers',
  'type': 'input_subsidy'}]

In [103]:
def generate_disease_solution(disease: str, disease_info: Dict, soil_info: Dict, budget: float) -> Dict:
    prompt = f"""You are an agricultural expert. Provide practical solutions for a farmer.

Disease: {disease}
Cause: {disease_info['cause']}
Soil Relation: {disease_info['soil_relation']}
Soil Type Characteristics: {soil_info['characteristics']}
Farmer's Budget: Rs. {budget}

Provide TWO solutions:
1. LOW-COST SOLUTION (for budget < 3000): Simple, affordable steps using local resources
2. HIGH-COST SOLUTION (for budget >= 3000): Advanced approach with better technology

Focus on:
- Soil amendments needed (specific to the deficiency)
- Disease treatment steps
- Prevention measures
- Cost-effective options within budget

Format:
LOW-COST:
[solution details]

HIGH-COST:
[solution details]
"""

    response = generate_text_with_llm(prompt, max_tokens=600)

    low_cost = []
    high_cost = []
    parts = response.split("HIGH-COST SOLUTION:")

    low_part = parts[0]
    high_part = parts[1] if len(parts) > 1 else ""

    low_body = low_part.split("LOW-COST SOLUTION:")[1].strip()
    high_body = high_part.strip()

    low_cost = [block.strip() for block in low_body.split("\n\n") if block.strip()]
    high_cost = [block.strip() for block in high_body.split("\n\n") if block.strip()]

    if not low_cost:
        low_cost = [
            f"Apply neem-based organic pesticide (Rs. 200-500)",
            f"Improve soil with compost and farmyard manure (Rs. 500-1000)",
            f"Ensure proper drainage to prevent moisture buildup",
            f"Remove and destroy infected plant parts",
            f"Use crop rotation to break disease cycle"
        ]

    if not high_cost:
        high_cost = [
            f"Apply systemic fungicide (Mancozeb/Copper oxychloride) - Rs. 800-1500",
            f"Soil amendment with specific nutrients: {soil_info['amendments']} - Rs. 2000-3000",
            f"Install drip irrigation for precise water management - Rs. 5000+",
            f"Use certified disease-resistant varieties next season",
            f"Implement integrated pest management (IPM) practices"
        ]

    return {
        "disease": disease,
        "cause": disease_info['cause'],
        "soil_relation": disease_info['soil_relation'],
        "low_cost_solution": low_cost,
        "high_cost_solution": high_cost
    }


In [104]:
# generate_disease_solution("rice_blast",disease_info,soil_info,3000)

{'disease': 'rice_blast',
 'cause': 'fungal infection (Magnaporthe oryzae)',
 'soil_relation': 'excessive nitrogen and poor silicon levels',
 'low_cost_solution': ["Soil Amendments:\n1. Use neem cake (50 kg/ha) as a soil amendment, as it contains nitrogen and improves soil structure, along with having fungicidal properties against rice blast.\n2. Apply rice bran (10-15 t/ha) to increase the soil's silicon level, which is essential in preventing rice blast.\n3. Mix calcium amendments, like lime or gypsum, to improve soil structure and make nutrients more available to the rice plant.\n4. Incorporate cow dung (10 t/ha) or compost (5 t/ha) for additional nutrients and organic matter to improve soil structure.",
  'Disease Treatment:\n1. Use neem seed extract (1-2 kg neem seeds/100 liters water) as a natural fungicide. Spray it on the rice fields at the onset and during the active stages of the rice blast disease.\n2. Apply copper sulfate (2-3 kg/ha) as a fungicide, which is effective again

In [106]:
def generate_weed_solution(budget: float) -> Dict:
    if budget <= 3000:
        return {
            "solution_type": "low_cost",
            "methods": WEED_MANAGEMENT["low_cost"],
            "estimated_cost": "Rs. 500 - 2000"
        }
    else:
        return {
            "solution_type": "high_cost",
            "methods": WEED_MANAGEMENT["high_cost"],
            "estimated_cost": "Rs. 3000 - 8000"
        }


In [107]:
# generate_weed_solution(3000)

{'solution_type': 'low_cost',
 'methods': ['Manual weeding using hand tools (khurpi, kassi)',
  'Mulching with crop residue or dried leaves',
  'Inter-cropping to suppress weed growth',
  'Timely hoeing between rows',
  'Use of black plastic mulch in small patches'],
 'estimated_cost': 'Rs. 500 - 2000'}

In [113]:
def generate_soil_improvement_plan(soil_info: Dict, budget: float) -> Dict:
    prompt = f"""You are a soil scientist. Create a practical soil improvement plan.

Soil Characteristics: {soil_info['characteristics']}
Recommended Amendments: {soil_info['amendments']}
Farmer's Budget: Rs. {budget}

Provide specific steps for soil improvement with estimated costs.

Format as numbered steps with cost ranges."""

    response = generate_text_with_llm(prompt, max_tokens=400)
    recommendations = [s.strip() for s in response.split("\n")
                   if s.strip() and any(c.isdigit() for c in s)]

    if not recommendations:
        if budget < 3000:
            recommendations = [
                "Soil Testing: Perform a soil test to check pH and nutrients (around Rs. 500).",
                "Adding Organic Matter: Apply about 5 tons/acre of compost or FYM (around Rs. 300–600).",
                "Liming: Add lime (e.g., CaCO3) as per soil test to reach pH 6.0–7.0 (around Rs. 1,200 for ~4 tons).",
                "Fertilization: Apply balanced NPK fertilizer, e.g., 100 kg mix at ~Rs. 4.5/kg ≈ Rs. 450, adjusted to soil-test recommendation."
            ]
        else:
            recommendations = [
                "Soil Testing: Detailed lab soil test and consultation (around Rs. 500 or more).",
                "Adding Organic Matter: Higher-quality compost/FYM and bio-organics at recommended rates.",
                "Liming: Use good-quality agricultural lime based on soil test to keep pH 6.0–7.0.",
                "Fertilization: Use soil-test-based NPK with premium formulations and micronutrients."
            ]

    return {
        "recommendations": recommendations,
        "budget_category": "low_cost" if budget <= 3000 else "high_cost"
    }



In [114]:
# generate_soil_improvement_plan(soil_info,3000)

{'recommendations': ["1. Soil Testing: Before making any amendments, it's essential to determine the current soil pH and nutrient levels. Submit a soil sample to a laboratory for analysis, cost may range between Rs. 300 - Rs. 500.",
  '2. Organic Matter Addition: Adding organic matter will improve the structure and fertility of the soil. Mix well 5-10 tons (approximately 30 m³) of farmyard manure or compost, cost between Rs. 600 - Rs. 1000, considering transportation and labor costs.',
  '3. Lime Application: Since the soil is acidic, apply lime to increase pH. Use Agro lime @ 5 tons per hectare (approximately 1 ton/4300 sq ft.), cost around Rs. 1500 - Rs. 2500.',
  '4. Fertilizer Application: Each year, apply NPK fertilizers to correct the low nitrogen and phosphorus levels. Use a 16:16:16 NPK granular fertilizer, at a rate of 50 kg per hectare (approximately 12-13 kg per acre), cost around Rs. 1500 - Rs. 2000.',
  '5. Crop Selection: Based on soil test results, choose crops that grow

In [18]:
def create_action_plan(user_input: Dict) -> Dict:
    disease_info = None
    disease_solution = None

    if user_input["disease"]:
        print(f"\nAnalyzing disease: {user_input['disease']}")
        disease_info = retrieve_disease_context(user_input["disease"])
        print(f"Disease cause: {disease_info['cause']}")
        print(f"Soil relation: {disease_info['soil_relation']}")

    print(f"\nAnalyzing soil type: {user_input['soil_type']}")
    soil_info = retrieve_soil_context(user_input["soil_type"])

    print(f"\nRetrieving government schemes for: {user_input['state']}")
    govt_schemes = get_government_schemes(user_input["state"])

    action_plan = {
        "input_data": user_input,
        "analysis": {
            "soil_analysis": soil_info,
            "disease_analysis": disease_info
        },
        "recommendations": {}
    }

    # Disease solution (if applicable)
    if user_input["disease"] and disease_info:
        print(f"\nGenerating disease treatment plan...")
        disease_solution = generate_disease_solution(
            user_input["disease"],
            disease_info,
            soil_info,
            user_input["budget"]
        )
        action_plan["recommendations"]["disease_management"] = disease_solution

    # Weed solution
    if user_input["weed_detected"]:
        print(f"\nGenerating weed management plan...")
        weed_solution = generate_weed_solution(user_input["budget"])
        action_plan["recommendations"]["weed_management"] = weed_solution

    # Soil improvement
    print(f"\nGenerating soil improvement plan...")
    soil_plan = generate_soil_improvement_plan(soil_info, user_input["budget"])
    action_plan["recommendations"]["soil_improvement"] = soil_plan

    # Budget-based plan selection
    if user_input["budget"] < 3000:
        plan_type = "Simple, Affordable Steps (Low-Cost Approach)"
        action_plan["selected_plan"] = "low_cost"
    else:
        plan_type = "Best-in-Class Approach (Advanced Technology)"
        action_plan["selected_plan"] = "high_cost"

    action_plan["plan_type"] = plan_type
    action_plan["government_schemes"] = govt_schemes

    # Generate final summary
    print(f"\nGenerating final recommendations summary...")
    summary_prompt = f"""Summarize the following agricultural action plan in 3-4 clear points:

Budget: Rs. {user_input['budget']}
Plan Type: {plan_type}
Soil Type: {user_input['soil_type']}
Disease: {user_input['disease'] if user_input['disease'] else 'None'}
Weed Present: {'Yes' if user_input['weed_detected'] else 'No'}

Key Actions Needed:
- Soil improvement: {soil_info['amendments']}
- Disease treatment: {disease_info['cause'] if disease_info else 'Not applicable'}
- Weed management: {'Required' if user_input['weed_detected'] else 'Not needed'}

Provide 3-4 actionable priority steps."""

    summary = generate_text_with_llm(summary_prompt, max_tokens=300)
    action_plan["executive_summary"] = summary

    return action_plan

In [19]:
def save_to_json(data: Dict, filename: str = None) -> str:
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"action_plan_{timestamp}.json"

    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

    return filename

In [20]:
def display_action_plan(action_plan: Dict):
    print(f"\nState: {action_plan['input_data']['state'].upper()}")
    print(f"Budget: Rs. {action_plan['input_data']['budget']}")
    print(f"Plan Type: {action_plan['plan_type']}")
    print(f"Input Mode: {action_plan['input_data']['input_mode']}")

    print(action_plan['executive_summary'])

    if "disease_management" in action_plan["recommendations"]:
        dm = action_plan["recommendations"]["disease_management"]
        print("\n" + "-" * 60)
        print("DISEASE MANAGEMENT")
        print("-" * 60)
        print(f"Disease: {dm['disease']}")
        print(f"Cause: {dm['cause']}")
        print(f"Soil Relation: {dm['soil_relation']}")

        if action_plan['selected_plan'] == 'low_cost':
            print("\nLOW-COST SOLUTION:")
            for i, step in enumerate(dm['low_cost_solution'], 1):
                print(f"  {i}. {step}")
        else:
            print("\nHIGH-COST SOLUTION:")
            for i, step in enumerate(dm['high_cost_solution'], 1):
                print(f"  {i}. {step}")

    if "weed_management" in action_plan["recommendations"]:
        wm = action_plan["recommendations"]["weed_management"]
        print("\n" + "-" * 60)
        print("WEED MANAGEMENT")
        print("-" * 60)
        print(f"Approach: {wm['solution_type'].upper()}")
        print(f"Estimated Cost: {wm['estimated_cost']}")
        print("\nMethods:")
        for i, method in enumerate(wm['methods'], 1):
            print(f"  {i}. {method}")

    if "soil_improvement" in action_plan["recommendations"]:
        si = action_plan["recommendations"]["soil_improvement"]
        print("\n" + "-" * 60)
        print("SOIL IMPROVEMENT PLAN")
        print("-" * 60)
        print(f"Budget Category: {si['budget_category'].upper()}")
        print("\nRecommendations:")
        for i, rec in enumerate(si['recommendations'], 1):
            print(f"  {i}. {rec}")

    print("\n" + "-" * 60)
    print("APPLICABLE GOVERNMENT SCHEMES")
    print("-" * 60)
    for scheme in action_plan['government_schemes']:
        print(f"\n• {scheme['name']}")
        print(f"  Subsidy: {scheme['subsidy']}")
        print(f"  Type: {scheme['type'].replace('_', ' ').title()}")

    print("\n" + "=" * 60)

In [21]:
def main():
    user_input = get_user_input()
    action_plan = create_action_plan(user_input)
    display_action_plan(action_plan)
    filename = save_to_json(action_plan)
    print(f"\nAction plan saved to: {filename}")

In [22]:
def batch_process_predictions(predictions_file: str):
    try:
        with open(predictions_file, 'r') as f:
            predictions = json.load(f)

        results = []

        for idx, pred in enumerate(predictions, 1):
            print(f"\nProcessing prediction {idx}/{len(predictions)}...")

            # Format input
            user_input = {
                "state": pred.get("state", "unknown"),
                "budget": pred.get("budget", 2000),
                "weed_detected": pred.get("weed_detected", False),
                "soil_type": pred.get("soil_type", "alluvial_soil"),
                "disease": pred.get("disease", None),
                "input_mode": "automatic",
                "timestamp": datetime.now().isoformat()
            }

            # Generate plan
            action_plan = create_action_plan(user_input)
            results.append(action_plan)

            # Save individual result
            filename = save_to_json(action_plan, f"action_plan_{idx}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
            print(f" Saved: {filename}")

        # Save batch results
        batch_filename = f"batch_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        with open(batch_filename, 'w', encoding='utf-8') as f:
            json.dump(results, f, indent=2, ensure_ascii=False)

        print(f"\nBatch processing complete! Results saved to: {batch_filename}")

    except FileNotFoundError:
        print(f"Error: File '{predictions_file}' not found!")
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in '{predictions_file}'")


In [23]:
def interactive_mode():
    while True:
        main()

        print("\n" + "=" * 60)
        continue_choice = input("Do you want to generate another plan? (yes/no): ").lower().strip()
        if continue_choice != "yes":
            print("\n👋 Thank you for using the Agricultural Recommendation System!")
            break

def api_mode(input_data: Dict) -> Dict:
    return create_action_plan(input_data)

In [25]:
EXTENDED_DISEASE_KB = {
    "potato_late_blight": {
        "cause": "Phytophthora infestans, favored by cool moist conditions",
        "soil_relation": "poor drainage and excess nitrogen",
        "symptoms": "water-soaked lesions, white fungal growth"
    },
    "grape_black_rot": {
        "cause": "Guignardia bidwellii fungal infection",
        "soil_relation": "calcium and potassium deficiency",
        "symptoms": "brown lesions with black dots"
    },
    "cotton_bacterial_blight": {
        "cause": "Xanthomonas bacteria",
        "soil_relation": "nitrogen excess and poor pH",
        "symptoms": "angular leaf spots, boll rot"
    },
    "wheat_leaf_rust": {
        "cause": "Puccinia triticina fungus",
        "soil_relation": "phosphorus deficiency",
        "symptoms": "orange pustules on leaves"
    },
    "maize_leaf_blight": {
        "cause": "Exserohilum turcicum fungus",
        "soil_relation": "nitrogen imbalance",
        "symptoms": "long tan lesions with dark borders"
    },
    "soybean_rust": {
        "cause": "Phakopsora pachyrhizi fungus",
        "soil_relation": "potassium deficiency",
        "symptoms": "small tan to dark brown lesions"
    },
    "apple_scab": {
        "cause": "Venturia inaequalis fungus",
        "soil_relation": "poor soil aeration",
        "symptoms": "olive-green spots on leaves and fruit"
    },
    "citrus_canker": {
        "cause": "Xanthomonas citri bacteria",
        "soil_relation": "copper and zinc deficiency",
        "symptoms": "raised brown lesions with yellow halos"
    }
}
DISEASE_KNOWLEDGE_BASE.update(EXTENDED_DISEASE_KB)


In [26]:
def semantic_search_disease(query: str, top_k: int = 3) -> List[tuple]:
    query_embedding = get_embeddings(query)

    if not query_embedding:
        return []

    results = []
    for disease_name, disease_info in DISEASE_KNOWLEDGE_BASE.items():
        # Create searchable text
        search_text = f"{disease_name} {disease_info['cause']} {disease_info['symptoms']}"
        doc_embedding = get_embeddings(search_text)

        if doc_embedding:
            # Calculate similarity score
            score = sum(a*b for a, b in zip(query_embedding[:len(doc_embedding)], doc_embedding))
            results.append((disease_name, disease_info, score))

    # Sort by score and return top_k
    results.sort(key=lambda x: x[2], reverse=True)
    return results[:top_k]

def enhanced_disease_diagnosis(symptoms_description: str) -> Dict:
    matches = semantic_search_disease(symptoms_description, top_k=3)

    if not matches:
        return {
            "disease": "unknown",
            "cause": "insufficient information",
            "soil_relation": "general soil health",
            "confidence": "low"
        }

    # Get best match
    best_match = matches[0]
    disease_name = best_match[0]
    disease_info = best_match[1]
    confidence = best_match[2]

    # Generate detailed diagnosis using LLM
    prompt = f"""You are a plant pathologist. Analyze these symptoms and provide diagnosis.

Symptoms Described: {symptoms_description}

Most Likely Disease: {disease_name}
Known Cause: {disease_info['cause']}
Known Symptoms: {disease_info['symptoms']}

Provide:
1. Confidence level (High/Medium/Low)
2. Brief explanation of why this diagnosis fits
3. Any alternative possibilities

Keep response under 100 words."""

    diagnosis_text = generate_text_with_llm(prompt, max_tokens=200)

    return {
        "disease": disease_name,
        "cause": disease_info['cause'],
        "soil_relation": disease_info['soil_relation'],
        "symptoms": disease_info['symptoms'],
        "confidence": "high" if confidence > 0.7 else "medium" if confidence > 0.4 else "low",
        "diagnosis_explanation": diagnosis_text,
        "alternative_matches": [(m[0], round(m[2], 2)) for m in matches[1:]]
    }

def generate_personalized_recommendation(user_input: Dict, action_plan: Dict) -> str:
    """Generate personalized farmer-friendly recommendation using LLM"""

    prompt = f"""You are talking to a farmer. Create a simple, actionable message in friendly language.

Farmer's Situation:
- Location: {user_input['state']}
- Budget: Rs. {user_input['budget']}
- Soil Type: {user_input['soil_type']}
- Weed Issue: {'Yes' if user_input['weed_detected'] else 'No'}
- Disease: {user_input['disease'] if user_input['disease'] else 'None detected'}

Selected Plan: {action_plan['plan_type']}

Write a short, encouraging message (5-6 sentences) that:
1. Acknowledges their situation
2. Highlights the top 2-3 priority actions
3. Mentions government support available
4. Gives realistic timeline for improvement

Use simple language, avoid technical jargon."""

    return generate_text_with_llm(prompt, max_tokens=300)

In [115]:
def main():
    user_input = get_user_input()
    action_plan = create_action_plan(user_input)
    display_action_plan(action_plan)
    filename = save_to_json(action_plan)
    print(f"\nAction plan saved to: {filename}")

In [117]:
main()


Enter your state (e.g., punjab, karnataka): karnataka
Enter your budget (in INR): 3000

Is this input from automatic prediction? (yes/no)
no

--- Manual Input Mode ---
Is weed present? (yes/no): yes
Enter soil type (red_soil/black_soil/alluvial_soil/laterite_soil/desert_soil/mountain_soil): red_soil
Enter disease name (or 'none' if no disease): rice_blast

Analyzing disease: rice_blast
Disease cause: fungal infection (Magnaporthe oryzae)
Soil relation: excessive nitrogen and poor silicon levels

Analyzing soil type: red_soil

Retrieving government schemes for: karnataka

Generating disease treatment plan...

Generating weed management plan...

Generating soil improvement plan...

Generating final recommendations summary...

AGRICULTURAL ACTION PLAN

State: KARNATAKA
Budget: Rs. 3000.0
Plan Type: Best-in-Class Approach (Advanced Technology)
Input Mode: manual

------------------------------------------------------------
EXECUTIVE SUMMARY
------------------------------------------------